** Analysis

* Time series visualization for stock (historical analysis - final value and moving averages)
* Overview details on page (summary and 3 ratios) 
* Annual Report details on page (10 items total - ratio calculation from here)
* Correlation analysis with GDP, oil prices, interest rates
* Data preparation

* Docker compose for postgresql and airflow(pandas - powerbi)

## Data Preparation

In [59]:
import requests
import pandas as pd
import json
import numpy as np

api_key = 'VRS4SWEZ6AC9K0Z5'


In [ ]:
stocks = {
    'IBM' : 1,
    'GOOG': 2, 
    'NVDA': 3,
    'ASML': 4,
    'AMD': 5    
}

In [81]:
class ETL:
    def __init__(self, stocks, api_key):
        self.stocks = stocks
        self.api_key = api_key
        self.uri_dict = {
            'daily' : 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=%s&apikey=%s',
            'info' : 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=%s&apikey=%s',
            'balance_sheet' : 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=%s&apikey=%s',
            'income_statement' : 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=%s&apikey=%s'
        }
            
    def get_prices(stock, stock_id):
        url = self.uri_dict['daily'] % (stock, self.api_key)
        r = requests.get(url)
        data = r.json()

        df = pd.DataFrame(data['Time Series (Daily)'], dtype='float').T
        df.columns = [col[3:].replace(' ','_') for col in df.columns]
        df.sort_index(inplace=True, ascending=False)
        df['stock_id'] = stock_id
        df['date']= pd.to_datetime(df.index)
        df.reset_index(drop=True, inplace=True)
        return df

    def get_info(stock, stock_id):
        url = self.uri_dict['info'] % (stock, self.api_key)
        r = requests.get(url)
        data = r.json()
        return data

    def get_financial(stock, stock_id, key='balance_sheet'):
        url = self.uri_dict[key] % (stock, self.api_key)
        r = requests.get(url)
        data = r.json()

        df['stock_id'] = 1
        df = df.replace('None', np.nan)
        for col in df_bs.columns:
            if col == 'fiscalDateEnding':
                df_bs[col] = pd.to_datetime(df_bs[col])
            elif col != 'reportedCurrency':
                print(col)
                df_bs[col] = pd.to_numeric(df_bs[col])
        return df

In [4]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
stock = 'IBM'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={stock}&apikey={api_key}'
r = requests.get(url)
data = r.json()

In [20]:
df = pd.DataFrame(data['Time Series (Daily)'], dtype='float').T
df.columns = [col[3:].replace(' ','_') for col in df.columns]
df.sort_index(inplace=True, ascending=False)
df['stock_id'] = 1
df['date']= pd.to_datetime(df.index)
df.reset_index(drop=True, inplace=True)
df

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,stock_id,date
0,123.11,124.100,122.8050,123.65,123.650000,4971936.0,0.0,1.0,1,2023-05-05
1,123.03,123.520,121.7563,122.57,122.570000,4468237.0,0.0,1.0,1,2023-05-04
2,125.46,125.570,123.2600,123.45,123.450000,4554212.0,0.0,1.0,1,2023-05-03
3,126.30,126.450,123.2700,125.16,125.160000,4445283.0,0.0,1.0,1,2023-05-02
4,126.35,126.750,126.0600,126.09,126.090000,2724992.0,0.0,1.0,1,2023-05-01
...,...,...,...,...,...,...,...,...,...,...
95,141.25,141.915,138.9700,140.16,138.451994,9834427.0,0.0,1.0,1,2022-12-16
96,148.52,148.980,141.5800,142.36,140.625185,6687744.0,0.0,1.0,1,2022-12-15
97,150.47,151.910,148.4500,149.86,148.033789,4205878.0,0.0,1.0,1,2022-12-14
98,150.37,153.210,149.9500,150.57,148.735137,8811553.0,0.0,1.0,1,2022-12-13


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   open               100 non-null    float64       
 1   high               100 non-null    float64       
 2   low                100 non-null    float64       
 3   close              100 non-null    float64       
 4   adjusted_close     100 non-null    float64       
 5   volume             100 non-null    float64       
 6   dividend_amount    100 non-null    float64       
 7   split_coefficient  100 non-null    float64       
 8   stock_id           100 non-null    int64         
 9   date               100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 7.9 KB


In [22]:
stock = 'IBM'
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={stock}&apikey={api_key}'
r = requests.get(url)
data = r.json()
print(data)

{'Symbol': 'IBM', 'AssetType': 'Common Stock', 'Name': 'International Business Machines', 'Description': 'International Business Machines Corporation (IBM) is an American multinational technology company headquartered in Armonk, New York, with operations in over 170 countries. The company began in 1911, founded in Endicott, New York, as the Computing-Tabulating-Recording Company (CTR) and was renamed International Business Machines in 1924. IBM is incorporated in New York. IBM produces and sells computer hardware, middleware and software, and provides hosting and consulting services in areas ranging from mainframe computers to nanotechnology. IBM is also a major research organization, holding the record for most annual U.S. patents generated by a business (as of 2020) for 28 consecutive years. Inventions by IBM include the automated teller machine (ATM), the floppy disk, the hard disk drive, the magnetic stripe card, the relational database, the SQL programming language, the UPC barcod

In [24]:
pd.DataFrame([data,data])

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,IBM,Common Stock,International Business Machines,International Business Machines Corporation (I...,51143,NYSE,USD,USA,TECHNOLOGY,COMPUTER & OFFICE EQUIPMENT,...,2.969,25.81,0.852,151.35,112.8,127.52,133.27,908045000,2023-06-10,2023-05-09
1,IBM,Common Stock,International Business Machines,International Business Machines Corporation (I...,51143,NYSE,USD,USA,TECHNOLOGY,COMPUTER & OFFICE EQUIPMENT,...,2.969,25.81,0.852,151.35,112.8,127.52,133.27,908045000,2023-06-10,2023-05-09


In [27]:
stock = 'IBM'
url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={stock}&apikey={api_key}'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2022-12-31', 'reportedCurrency': 'USD', 'totalAssets': '127243000000', 'totalCurrentAssets': '29118000000', 'cashAndCashEquivalentsAtCarryingValue': '7886000000', 'cashAndShortTermInvestments': '7886000000', 'inventory': '1552000000', 'currentNetReceivables': '14209000000', 'totalNonCurrentAssets': '96874000000', 'propertyPlantEquipment': '5334000000', 'accumulatedDepreciationAmortizationPPE': '13361000000', 'intangibleAssets': '67133000000', 'intangibleAssetsExcludingGoodwill': '11184000000', 'goodwill': '55949000000', 'investments': 'None', 'longTermInvestments': '142000000', 'shortTermInvestments': '852000000', 'otherCurrentAssets': '2610000000', 'otherNonCurrentAssets': 'None', 'totalLiabilities': '105222000000', 'totalCurrentLiabilities': '31505000000', 'currentAccountsPayable': '4051000000', 'deferredRevenue': '15531000000', 'currentDebt': '9511000000', 'shortTermDebt': '4760000000', 'totalNonCurrentLiabilities': '83414000

In [43]:
df_bs = pd.DataFrame(data['annualReports'])

In [63]:
df_bs['stock_id'] = 1
df_bs = df_bs.replace('None', np.nan)
for col in df_bs.columns:
    if col == 'fiscalDateEnding':
        df_bs[col] = pd.to_datetime(df_bs[col])
    elif col != 'reportedCurrency':
        print(col)
        df_bs[col] = pd.to_numeric(df_bs[col])
df_bs.dtypes

grossProfit
totalRevenue
costOfRevenue
costofGoodsAndServicesSold
operatingIncome
sellingGeneralAndAdministrative
researchAndDevelopment
operatingExpenses
investmentIncomeNet
netInterestIncome
interestIncome
interestExpense
nonInterestIncome
otherNonOperatingIncome
depreciation
depreciationAndAmortization
incomeBeforeTax
incomeTaxExpense
interestAndDebtExpense
netIncomeFromContinuingOperations
comprehensiveIncomeNetOfTax
ebit
ebitda
netIncome
stock_id


fiscalDateEnding                     datetime64[ns]
reportedCurrency                             object
grossProfit                                   int64
totalRevenue                                  int64
costOfRevenue                                 int64
costofGoodsAndServicesSold                    int64
operatingIncome                               int64
sellingGeneralAndAdministrative               int64
researchAndDevelopment                        int64
operatingExpenses                             int64
investmentIncomeNet                         float64
netInterestIncome                             int64
interestIncome                                int64
interestExpense                               int64
nonInterestIncome                           float64
otherNonOperatingIncome                       int64
depreciation                                  int64
depreciationAndAmortization                   int64
incomeBeforeTax                               int64
incomeTaxExp

In [58]:
df_bs.iloc[:, 2:]

,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,stock_id
0,32687000000,60530000000,27842000000,385000000,6408000000,18609000000,6567000000,26279000000,None,-1216000000,...,2395000000,1013000000,-626000000,1216000000,1783000000,8134000000,2229000000,4624000000,1639000000,1
1,31486000000,57350000000,25865000000,300000000,4786000000,18745000000,6488000000,26700000000,None,-1155000000,...,2529000000,5867000000,124000000,1155000000,4712000000,10582000000,7022000000,9551000000,5743000000,1
2,30865000000,55179000000,24314000000,439000000,4609000000,20561000000,6262000000,30966000000,None,-1288000000,...,2468000000,4230000000,-1360000000,1288000000,3932000000,4850000000,5518000000,7986000000,5590000000,1
3,31533000000,57714000000,26181000000,591000000,9004000000,18724000000,5910000000,27484000000,None,-1344000000,...,1850000000,9491000000,60000000,1344000000,7146000000,10324000000,10835000000,12685000000,9431000000,1
4,36936000000,79591000000,42655000000,42655000000,10838000000,19366000000,5379000000,26098000000,None,-723000000,...,1353000000,11347000000,2619000000,723000000,8723000000,8252000000,12070000000,13423000000,8728000000,1


In [46]:
pd.to_datetime(df['fiscal'])

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,stock_id
0,2022-12-31,USD,32687000000,60530000000,27842000000,385000000,6408000000,18609000000,6567000000,26279000000,...,2395000000,1013000000,-626000000,1216000000,1783000000,8134000000,2229000000,4624000000,1639000000,1
1,2021-12-31,USD,31486000000,57350000000,25865000000,300000000,4786000000,18745000000,6488000000,26700000000,...,2529000000,5867000000,124000000,1155000000,4712000000,10582000000,7022000000,9551000000,5743000000,1
2,2020-12-31,USD,30865000000,55179000000,24314000000,439000000,4609000000,20561000000,6262000000,30966000000,...,2468000000,4230000000,-1360000000,1288000000,3932000000,4850000000,5518000000,7986000000,5590000000,1
3,2019-12-31,USD,31533000000,57714000000,26181000000,591000000,9004000000,18724000000,5910000000,27484000000,...,1850000000,9491000000,60000000,1344000000,7146000000,10324000000,10835000000,12685000000,9431000000,1
4,2018-12-31,USD,36936000000,79591000000,42655000000,42655000000,10838000000,19366000000,5379000000,26098000000,...,1353000000,11347000000,2619000000,723000000,8723000000,8252000000,12070000000,13423000000,8728000000,1


In [64]:
stock = 'IBM'
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={stock}&apikey={api_key}'
r = requests.get(url)
data = r.json()

print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2022-12-31', 'reportedCurrency': 'USD', 'grossProfit': '32687000000', 'totalRevenue': '60530000000', 'costOfRevenue': '27842000000', 'costofGoodsAndServicesSold': '385000000', 'operatingIncome': '6408000000', 'sellingGeneralAndAdministrative': '18609000000', 'researchAndDevelopment': '6567000000', 'operatingExpenses': '26279000000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-1216000000', 'interestIncome': '162000000', 'interestExpense': '1216000000', 'nonInterestIncome': '365000000', 'otherNonOperatingIncome': '443000000', 'depreciation': '2407000000', 'depreciationAndAmortization': '2395000000', 'incomeBeforeTax': '1013000000', 'incomeTaxExpense': '-626000000', 'interestAndDebtExpense': '1216000000', 'netIncomeFromContinuingOperations': '1783000000', 'comprehensiveIncomeNetOfTax': '8134000000', 'ebit': '2229000000', 'ebitda': '4624000000', 'netIncome': '1639000000'}, {'fiscalDateEnding': '2021-12-31', 'reportedCurren

In [68]:
df_inc = pd.DataFrame(data['annualReports'])
df_inc['stock_id'] = 1
df_inc = df_bs.replace('None', np.nan)
for col in df_inc.columns:
    if col == 'fiscalDateEnding':
        df_inc[col] = pd.to_datetime(df_inc[col])
    elif col != 'reportedCurrency':
        print(col)
        df_inc[col] = pd.to_numeric(df_inc[col])
df_inc.dtypes

grossProfit
totalRevenue
costOfRevenue
costofGoodsAndServicesSold
operatingIncome
sellingGeneralAndAdministrative
researchAndDevelopment
operatingExpenses
investmentIncomeNet
netInterestIncome
interestIncome
interestExpense
nonInterestIncome
otherNonOperatingIncome
depreciation
depreciationAndAmortization
incomeBeforeTax
incomeTaxExpense
interestAndDebtExpense
netIncomeFromContinuingOperations
comprehensiveIncomeNetOfTax
ebit
ebitda
netIncome
stock_id


fiscalDateEnding                     datetime64[ns]
reportedCurrency                             object
grossProfit                                   int64
totalRevenue                                  int64
costOfRevenue                                 int64
costofGoodsAndServicesSold                    int64
operatingIncome                               int64
sellingGeneralAndAdministrative               int64
researchAndDevelopment                        int64
operatingExpenses                             int64
investmentIncomeNet                         float64
netInterestIncome                             int64
interestIncome                                int64
interestExpense                               int64
nonInterestIncome                           float64
otherNonOperatingIncome                       int64
depreciation                                  int64
depreciationAndAmortization                   int64
incomeBeforeTax                               int64
incomeTaxExp

In [69]:
df_inc

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,stock_id
0,2022-12-31,USD,32687000000,60530000000,27842000000,385000000,6408000000,18609000000,6567000000,26279000000,...,2395000000,1013000000,-626000000,1216000000,1783000000,8134000000,2229000000,4624000000,1639000000,1
1,2021-12-31,USD,31486000000,57350000000,25865000000,300000000,4786000000,18745000000,6488000000,26700000000,...,2529000000,5867000000,124000000,1155000000,4712000000,10582000000,7022000000,9551000000,5743000000,1
2,2020-12-31,USD,30865000000,55179000000,24314000000,439000000,4609000000,20561000000,6262000000,30966000000,...,2468000000,4230000000,-1360000000,1288000000,3932000000,4850000000,5518000000,7986000000,5590000000,1
3,2019-12-31,USD,31533000000,57714000000,26181000000,591000000,9004000000,18724000000,5910000000,27484000000,...,1850000000,9491000000,60000000,1344000000,7146000000,10324000000,10835000000,12685000000,9431000000,1
4,2018-12-31,USD,36936000000,79591000000,42655000000,42655000000,10838000000,19366000000,5379000000,26098000000,...,1353000000,11347000000,2619000000,723000000,8723000000,8252000000,12070000000,13423000000,8728000000,1


In [71]:
daily = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=%s&apikey=%s'

In [74]:
daily % ('IBM', 'DDKEY')

'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&apikey=DDKEY'